In [2]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
reviews_train = load_files("../data/aclImdb/train/")

In [4]:
text_train, y_train = reviews_train.data, reviews_train.target

In [5]:
type(text_train)

list

In [6]:
len(text_train)

25000

In [7]:
text_train[6]

b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."

In [8]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [9]:
text_train[6]

b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.  Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life.   I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."

In [10]:
np.bincount(y_train)

array([12500, 12500])

##### Now We Load The Test Dataset

In [11]:
reviews_test = load_files("../data/aclImdb/test/")

In [12]:
text_test, y_test = reviews_test.data, reviews_test.target

In [13]:
len(text_test)

25000

In [14]:
np.bincount(y_test)

array([12500, 12500])

In [15]:
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

##### Introduce `CountVectorizer()`

In [16]:
vect = CountVectorizer()

In [17]:
bards_words =["The fool doth think he is wise,",
             "but the wise man knows himself to be a fool"]

In [18]:
vect.fit(bards_words)

CountVectorizer()

In [19]:
len(vect.vocabulary_)

13

In [20]:
vect.vocabulary_

{'the': 9,
 'fool': 3,
 'doth': 2,
 'think': 10,
 'he': 4,
 'is': 6,
 'wise': 12,
 'but': 1,
 'man': 8,
 'knows': 7,
 'himself': 5,
 'to': 11,
 'be': 0}

In [21]:
bag_of_words = vect.transform(bards_words)

In [22]:
repr(bag_of_words)

"<2x13 sparse matrix of type '<class 'numpy.int64'>'\n\twith 16 stored elements in Compressed Sparse Row format>"